In [134]:
import pandas as pd
pd.options.display.max_rows = 50
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt

# api https://live.euronext.com/fr/intraday_chart/getDetailedQuoteAjax/FR0000130452-XPAR/full

In [135]:
df= pd.read_excel("dataParis.xlsx")
df.drop([0,1,2], inplace=True)
df.reset_index(drop=True, inplace=True)


In [200]:
def getMarketCap(ISIN):
    url = f"https://live.euronext.com/fr/intraday_chart/getDetailedQuoteAjax/{ISIN}-XPAR/full"
    response = requests.get(url)
    if(response.ok):
        soup = BeautifulSoup(response.text)
        try:
            capitalization = soup.find("tbody").findAll('tr')[11].find(class_="font-weight-medium").text
            return capitalization
        except IndexError:
            return 0

def convertToInt(ISIN):
    if(type(ISIN)==str):
        if("Md" in ISIN):
            return (float(ISIN.replace("Md","").replace(",",".")))
        if("Md" in ISIN):
            return 0

def getMarketCapInt(ISIN):
    return convertToInt(getMarketCap(ISIN))

In [201]:
def addMarketCap():
    for i, row in df.iterrows():
        if(i>106):
            print(row['ISIN'])
            df.at[i,'MarketCap'] = getMarketCapInt(row['ISIN'])
            print(i)
    df.sort_values(by='MarketCap', ascending=False,ignore_index=True, inplace=True)


In [231]:
def removeNonFrenchCompany():
    index=[]
    for i, row in df.iterrows():
        if(not('FR' in df.loc[i]['ISIN'])):
            index.append(i)
    df.drop(index, inplace=True)
    df.reset_index(drop=True, inplace=True)
removeNonFrenchCompany()

In [391]:
def pieByPercent(i,size=150):
    plt.rcParams['figure.dpi'] = size
    plt.rcParams['savefig.dpi'] = size
    name = df['Name'].to_list()
    marketCap = df['MarketCap'].to_list()
    plt.pie(marketCap[0:i], labels =name[0:i], wedgeprops={'edgecolor':'black', 'linewidth': 0.5}, shadow=True, autopct='%1.1f%%', textprops={'fontsize': 1500/size})
    plt.title(f"Largest {i} french companies listed on Euronext Paris")
    plt.savefig(f"piechart{i}")



    